In [1]:
#python_bitbankccのパッケージをインポート
import python_bitbankcc 
import datetime
import os 
import time
import numpy as np
import pandas as pd
import sys

In [2]:
# public API classのオブジェクトを取得
pub = python_bitbankcc.public()

# ティッカー情報を取得
value = pub.get_ticker( 'xrp_jpy' )
print(type(value))
#現在の売り注文の最安値
print('sell:' + value['sell'])
#現在の買い注文の最安値
print('buy:' + value['buy'])

<class 'dict'>
sell:66.200
buy:66.190


In [3]:
print(type(value))
ticker_inf=['現在の売り注文の最安値','現在の買い注文の最安値','過去24時間の最高値取引価格',
            '過去24時間の最安値取引価格','最新取引価格','過去24時間の出来高','なんかの時間']
data_name=[]
values = []
for key,val in value.items():
    data_name.append(key)
    values.append(val)
information = list(zip(data_name,values,ticker_inf))
for key,val,inf in information:
    print(f'{key} = {val} :{inf}')
print(datetime.datetime.fromtimestamp(value['timestamp']/1000))#これで表示できる
#print(information)

<class 'dict'>
sell = 66.200 :現在の売り注文の最安値
buy = 66.190 :現在の買い注文の最安値
high = 66.666 :過去24時間の最高値取引価格
low = 60.200 :過去24時間の最安値取引価格
last = 66.200 :最新取引価格
vol = 216220167.6352 :過去24時間の出来高
timestamp = 1537696628776 :なんかの時間
2018-09-23 18:57:08.776000


In [4]:
# value = pub.get_ticker( '' )
# pair = ['btc_jpy', 'xrp_jpy', 'ltc_btc', 'eth_btc', 'mona_jpy', 'mona_btc', 'bcc_jpy', 'bcc_btc']

In [5]:
# 全約定履歴を取得
records = pub.get_transactions( 'xrp_jpy' )
print(type(records))
# for key,val in records.items():
#     print(f'{key} = {val}')
#     print()
#約定履歴の一例
trans_ex = records['transactions'][0]
print('取引ID:' + str(trans_ex['transaction_id']))
print('売買情報:' + trans_ex['side'])
print('約定価格:' + trans_ex['price'])
print('数量:' + trans_ex['amount'])
print('約定日時:' + str(datetime.datetime.fromtimestamp(trans_ex['executed_at']/1000)))

<class 'dict'>
取引ID:19014574
売買情報:sell
約定価格:66.200
数量:3623.4554
約定日時:2018-09-23 18:57:08.389000


In [6]:
# ロウソク足データを取得
value = pub.get_candlestick( 'xrp_jpy',  '5min', '20180504' )

#ローソク足情報の一例(2018/5/4 午前9時の1時間足)
candle = value['candlestick'][0]
print('candle type:' + candle['type'])
print('始値：' + candle['ohlcv'][0][0])
print('高値：' + candle['ohlcv'][0][1])
print('安値：' + candle['ohlcv'][0][2])
print('終値：' + candle['ohlcv'][0][3])
print('出来高：' + candle['ohlcv'][0][4])
print('対象時刻：' + str(datetime.datetime.fromtimestamp(candle['ohlcv'][0][5]/1000)))

print(candle['ohlcv'][0][0])
print(datetime.datetime.fromtimestamp(candle['ohlcv'][-1][-1]/1000))
print(len(candle['ohlcv']))
print(type(candle['ohlcv']))
print(len(value['candlestick']))


candle type:5min
始値：95.899
高値：95.900
安値：94.800
終値：94.973
出来高：1184813.9545
対象時刻：2018-05-04 09:00:00
95.899
2018-05-05 08:55:00
288
<class 'list'>
1


In [7]:
# 使い方
# value = get_candlestick( 'ペア', 'ローソク足のタイプ', 'YYYYMMDD 型の日付' )
# ローソク足のタイプ例⇒1min, 5min, 15min, 30min, 1hour, 4hour, 8hour, 12hour, 1day, 1week
# YYYYMMDD 型(YYYY型でも良い)の日付例⇒20180504, 2017等

In [11]:
#ここで仮想通貨の2年分のtrain_test_dataを取得する　簡単のために5min足にする

def get_candle(trade_name,span,back_day):
    # ロウソク足データを取得
    value = pub.get_candlestick( trade_name,span, back_day )
    candle = value['candlestick'][0]
    #print(candle['ohlcv'][0])
    dataframe = pd.DataFrame(candle['ohlcv'],columns=["Open", "High","Low","Close","Volume","Timestamp"])
    return dataframe

def read_date(x):
    return datetime.datetime.fromtimestamp(x/1000)


df1 = get_candle( 'xrp_jpy',  '12hour', '2018')

df1['Timestamp'] = df1['Timestamp'].apply(read_date)
print(len(df1))
print(df1)

#今から指定した日までの足をとる
# def conect_data(name,time,day):
    
# pd.concat([df1, df2, df3])



531
        Open     High      Low    Close          Volume           Timestamp
0    244.434  244.978  231.000  238.001    2549414.0366 2018-01-01 09:00:00
1    238.001  249.999  220.000  243.000    3332084.5765 2018-01-01 21:00:00
2    243.500  248.000  236.300  244.999    3474777.4859 2018-01-02 09:00:00
3    244.999  267.999  243.895  258.612    5970917.9580 2018-01-02 21:00:00
4    258.569  292.900  256.783  283.000    8796858.5909 2018-01-03 09:00:00
5    283.000  329.999  276.000  326.898   10378646.0837 2018-01-03 21:00:00
6    326.897  401.000  324.000  379.500   15281944.1075 2018-01-04 09:00:00
7    379.500  397.000  322.123  358.873   10030005.6751 2018-01-04 21:00:00
8    358.999  378.500  322.000  352.002    9343377.3393 2018-01-05 09:00:00
9    352.002  355.000  300.000  312.428    8051291.9723 2018-01-05 21:00:00
10   312.000  335.000  306.001  325.000    4254586.8040 2018-01-06 09:00:00
11   324.980  348.700  300.000  323.139    3315351.9435 2018-01-06 21:00:00
12   323

In [ ]:
date = datetime.date.today()
day = '{year}{month:02}{day:02}'.format(year = date.year,month = date.month,day = date.day)
print(day)

# timestamp = ''
# for i in range(3):
#     timestamp += str(date[i])


In [ ]:
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)
yesterday = today - datetime.timedelta(days=12)
#today = datetime.datetime.strftime(today, '%Y%m%d')
print(today == (tomorrow -datetime.timedelta(days=1)),( datetime.datetime.strptime('20180921', '%Y%m%d')) != today)
print("tomorrow -> " + datetime.datetime.strftime(tomorrow, '%Y%m%d'))
print("yesterday -> " + datetime.datetime.strftime(yesterday, '%Y%m%d'))

if today < tomorrow:
    print('True')

In [ ]:

def make_df(trade_name,span,back_day,today):#back_day,todayはdatetime
    i = 1
    while back_day <= today - datetime.timedelta(days=1):
        if i == 1:
            df1 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y%m%d'))
            back_day = back_day + datetime.timedelta(days=1)
            #print(back_day,len(df1))
            i += 1
        else:
            df2 = get_candle(trade_name,span,datetime.datetime.strftime(back_day, '%Y%m%d'))
            df1 = pd.concat([df1, df2])
            back_day = back_day + datetime.timedelta(days=1)
            #print(back_day,len(df1))
            i += 1
    
    return df1

back_day = today - datetime.timedelta(days=10)
trade_name = 'xrp_jpy'
span = '1hour'
df1 =  make_df(trade_name,span,back_day,today)
print(df1.tail(),len(df1))